### Proyecto práctico

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, una solución óptima será alcanzada cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

   1) Implementar la red neuronal que se usará en la solución
    
   2) Implementar las distintas piezas de la solución DQN
    
   3) Justificar la respuesta en relación a los resultados obtenidos

IMPORTANTE:

- Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.

- Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).

- Tened en cuenta que las versiones de librerías recomendadas son Tensorflow==1.13.1, Keras==2.2.4 y keras-rl==0.4.2

In [ ]:
# Uncomment this line for installing keras-rl on Google collaboratory
#!pip install keras-rl

#### Importar librerías

In [12]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from keras.optimizers import Adam
import keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

In [ ]:
# para tensorboard
# Load the TensorBoard notebook extension
%load_ext tensorboard

import tensorflow as tf
import datetime, os

# Clear any logs from previous runs
!rm -rf ./logs/ 

#### Configuración base

In [13]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [14]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L') # a escala de grises
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8') # casting a 8 bits

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255. # normalizar entre 0 y 1
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.) # se acota para que no sea tan sensible a valores extremos

In [15]:
# Acciones disponibles en el entorno
nb_actions = env.action_space.n
nb_actions

6

In [16]:
# Nombres de las acciones
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [17]:
# Dimensions de las observaciones del entorno
env.observation_space.shape

(210, 160, 3)

1) Implementación de la red neuronal

In [19]:
# Next, we build our model. We use the same model that was described by Mnih et al. (2015).
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()

# segun el backend, se usa un ordenamiento de dimensiones diferente (para tensorflow o theano)
if K.image_dim_ordering() == 'tf':
    # (width, height, channels)
    model.add(Permute((2, 3, 1), input_shape=input_shape))
elif K.image_dim_ordering() == 'th':
    # (channels, width, height)
    model.add(Permute((1, 2, 3), input_shape=input_shape))
else:
    raise RuntimeError('Unknown image_dim_ordering.')

model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear')) # lineal porque son las recompensas esperadas a futuro para cada accion (nb_actions)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_3 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
activation_11 (Activation)   (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
activation_12 (Activation)   (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
activation_13 (Activation)   (None, 7, 7, 64)          0         
__________

2) Implementación de la solución DQN

In [20]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()

In [21]:
# nb_steps será el tiempo que durará la exploración, antes de pasar a explotación. 
# value_test guarda un pelín de aleatoriedad para minimizar el riesgo de estancamiento en minimos durante el test
# empieza muy aleatorio a 1 (100%) de acciones aleatorias y va decreciendo a más acciones 'predichas/aprendidas'
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps',
                              value_max=1., value_min=.1, value_test=.05,
                              nb_steps=1000000)

In [22]:
# recordar que el modelo se 'duplica' uno para target y otro para predicción, ambos idénticos
# gamma es el discount factor de las rewards
# target se actualiza cada 10000 steps
# cada 20 steps, se actualizan los pesos del modelo
# bajamos el learning rate
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy,
               memory=memory, processor=processor,
               nb_steps_warmup=50000, gamma=.99,
               target_model_update=10000,
               train_interval=20)
dqn.compile(Adam(lr=.00025), metrics=['mae'])

In [23]:
# Training part
# log en pantalla cada 10,000 steps
# cada 250000 steps almaceno versiones del modelo (callbacks)
# el log escrito, cada 100 steps
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(env_name)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]

dqn.fit(env, callbacks=callbacks, nb_steps=1750000, log_interval=10000, visualize=True)

dqn.save_weights(weights_filename, overwrite=True)

Training for 1750000 steps ...
Interval 1 (0 steps performed)


/Users/inigo/.local/share/virtualenvs/08_aprendizaje_por_refuerzo-a97gBoCq/lib/python3.7/site-packages/gym/envs/atari/environment.py:257: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  "We strongly suggest supplying `render_mode` when "


10000/10000 [==============================] - 35s 4ms/step - reward: 0.0138
14 episodes - episode_reward: 9.571 [5.000, 15.000] - lives: 2.107

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 39s 4ms/step - reward: 0.0126
14 episodes - episode_reward: 8.786 [2.000, 16.000] - lives: 2.132

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 35s 4ms/step - reward: 0.0139
15 episodes - episode_reward: 9.467 [4.000, 22.000] - lives: 2.050

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 35s 3ms/step - reward: 0.0127
14 episodes - episode_reward: 9.143 [4.000, 17.000] - lives: 2.085

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 33s 3ms/step - reward: 0.0155
13 episodes - episode_reward: 11.692 [5.000, 31.000] - lives: 2.078

Interval 6 (50000 steps performed)
   19/10000 [..............................] - ETA: 33s - reward: 0.0000e+00 WARNING:tensorflow:Fro

In [25]:
# Testing part to calculate the mean reward
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 4.000, steps: 659
Episode 2: reward: 7.000, steps: 699
Episode 3: reward: 5.000, steps: 679
Episode 4: reward: 6.000, steps: 516
Episode 5: reward: 10.000, steps: 974
Episode 6: reward: 3.000, steps: 586
Episode 7: reward: 7.000, steps: 628
Episode 8: reward: 7.000, steps: 876
Episode 9: reward: 7.000, steps: 600
Episode 10: reward: 9.000, steps: 836


In [ ]:
%tensorboard --logdir logs/fit

3) Justificación de los parámetros seleccionados y de los resultados obtenidos